In [86]:
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact
import random
%matplotlib inline


import ipywidgets as widgets

central_time = 50


# idea: households have locations, people have locations near their household
num_households = 9
household_sizes = [2,3,4]
householdGroups = {}
node_locations = {}
setOfChildren = []

# print('early set of children ' + str(sorted(setOfChildren)))

circle = nx.cycle_graph(num_households)
circle_pos = nx.spring_layout(circle)

for household in circle_pos:
  size = random.choice(household_sizes)
  householdGroups[household] = []
  for i in range(size):
    householdGroups[household].append((household,i))
    locationX = circle_pos[household][0] + random.uniform(-0.2, 0.2)
    locationY = circle_pos[household][1] + random.uniform(-0.2, 0.2)
    node_locations[(household, i)] = (locationX, locationY)

allgraph = nx.Graph()
for (h, i) in node_locations:
  allgraph.add_node((h, i))
  if i == 0:
      setOfChildren.append((h, i))

for household in householdGroups:
  for i in range(len(householdGroups[household])):
    for j in range(len(householdGroups[household])):
      allgraph.add_edge(householdGroups[household][i], householdGroups[household][j])


cccGroups = {}
cccSize = 3
numberCCC = 3

for i in range(numberCCC):
  cccGroups[i] = []
  for j in range(cccSize):
    for household in householdGroups:
      if  household >= (i)*cccSize and household < (i+1)*cccSize:
        cccGroups[i].append(household)
  cccGroups[i] = list(set(cccGroups[i]))

def get_list_extra_edges(working_graph, total):
    extra_edges = []
    nodes = list(working_graph.nodes())
    for i in range(total):
        u = random.choice(nodes)
        v = random.choice(nodes)
        extra_edges.append((u ,v))
    return extra_edges



def make_ccc_graph(graph, cccGroups, childList):
  newGraph = nx.Graph()
  for (u, v) in graph.edges():
    newGraph.add_edge(u, v)
  for ccc in cccGroups:
    allThePeople = []
    for household in cccGroups[ccc]:
      allThePeople.extend(householdGroups[household])
    for person in allThePeople:
      if person in childList:
        for other in allThePeople:
          newGraph.add_edge(person, other)
  return newGraph


def make_school_graph(graph,  childList):
  newGraph = nx.Graph()
  for (u, v) in graph.edges():
    newGraph.add_edge(u, v)
    for child in childList:
      for other in childList:
        newGraph.add_edge(child, other)
  # half = len(householdGroups)/2
  # firstGroup = []
  # secondGroup = []
  # for household in householdGroups:
  #   for person in householdGroups[household]:
  #     if household < half and person in childList:
  #       firstGroup.append(person)
  #     elif household >= half and person in childList:
  #       secondGroup.append(person)
  #   for person in firstGroup:
  #     for other in firstGroup:
  #       newGraph.add_edge(person, other)
  #   for person in secondGroup:
  #     for other in secondGroup:
  #       newGraph.add_edge(person, other) 
  return newGraph

def runSimpleDisease (graph, probability):  
        timeHorizon = central_time
        timeToInfected = {}
        infected = [sorted(list(graph.nodes()))[0]]
        for time in range(timeHorizon):
            timeToInfected [time] = []
            for guy in infected:
                timeToInfected [time].append(guy)
            newInfected = []
            
            for guy in infected:
                neighbours = graph.neighbors(guy)
                for neigh in neighbours:
                    if neigh not in infected:
                        probabilityOfInfection = probability
                        thisLuck = random.random()
                        if thisLuck <= probabilityOfInfection:
                           newInfected.append(neigh)
                   
            newInfected = list(set(newInfected))
            infected.extend(newInfected)
            
            
        return timeToInfected

cccGraph = make_ccc_graph(allgraph, cccGroups, setOfChildren)

schoolGraph = make_school_graph(allgraph,  setOfChildren)

original_edges = allgraph.edges()
max_extra = 20
extra_edges = get_list_extra_edges(allgraph, max_extra)



extraEdges = range(max_extra)

extra_edges = get_list_extra_edges(allgraph, max_extra)

baseGraphDict = {'Lockdown':allgraph, 'CCC':cccGraph, 'School':schoolGraph}
dictOfSettingToOutcome = {}
dictOfSettingToGraph = {}

for base in baseGraphDict:
  dictOfSettingToOutcome[base] = {}
  dictOfSettingToGraph[base] = {}
  for extra_num in extraEdges:
     newGraph = nx.Graph()
     for (u, v) in baseGraphDict[base].edges():
       newGraph.add_edge(u,v)
     for i in range(extra_num):
       newGraph.add_edge(extra_edges[i][0], extra_edges[i][1])
    # make the graph from the base plus the extra edges
     dictOfSettingToGraph[base][extra_num] = newGraph
     outcome = runSimpleDisease (dictOfSettingToGraph[base][extra_num], 0.3)
     
     dictOfSettingToOutcome[base][extra_num] = outcome






def plot_the_graph(contact_type, number_extra_edges=0, time=0):
    # if contact_type == 'Lockdown':
    #  nx.draw(allgraph, pos = node_locations, alpha = 0.3)
    # #  nx.draw_networkx_nodes(allgraph, pos = node_locations, node_size)
    # #  nx.draw_networkx_edges(graph, pos = positions, edgelist=original_edges, width = 3, edge_color='dodgerblue')
    # elif contact_type == "CCC":
    #   nx.draw(cccGraph, pos = node_locations, alpha = 0.3)
    # elif contact_type == 'School':
    #   nx.draw(schoolGraph, pos = node_locations, alpha = 0.3)
    # else:
    #   nx.draw(allgraph, pos = node_locations, alpha = 0.3)
    nx.draw(dictOfSettingToGraph[contact_type][number_extra_edges], pos = node_locations,  alpha = 0.3)
    nx.draw_networkx_nodes(allgraph, pos = node_locations, nodelist = dictOfSettingToOutcome[contact_type][number_extra_edges][time], node_color='red')
    nx.draw_networkx_edges(allgraph, pos = node_locations, edgelist=original_edges, width = 3, edge_color='dodgerblue')
    plt.show() 




interact(plot_the_graph, contact_type={
             'Lockdown': 'Lockdown',
             'Closed-Group Childcare': 'CCC',
             'School Attendance': 'School',
         }, number_extra_edges=(0,max_extra), time=(0,central_time-1));
   




interactive(children=(Dropdown(description='contact_type', options={'Lockdown': 'Lockdown', 'Closed-Group Chil…